### 1- IMPORT LIBRARIES & DATA

In [1]:
# !pip install dash
# !pip install pandas 
# !pip install dash-bootstrap-components
# !pip install pymysql
# !pip install Flask-Caching 
# !pip install mysqlclient
# !pip install dash-extensions
# !pip install mysql-connector-python
# !pip install dash-auth==1.3.2

In [2]:
import os
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import pycountry
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_auth
from dash import dash_table
from dash.dash_table.Format import Group
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State, ClientsideFunction, MATCH, ALL, ClientsideFunction, Output, Input
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
#from navbar_tabs_layout import app_layout
import time
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta
import mysql.connector
import pymysql
pymysql.install_as_MySQLdb()
from flask_caching import Cache
#from dash_extensions.enrich import DashProxy, Output, Input, State, ServersideOutput,ServersideOutputTransform
#from app import app, cache
from credentials import VALID_USERNAME_PASSWORD_PAIRS
import webbrowser as web
from threading import Timer
from pandasai import PandasAI
from pandasai.llm.openai import OpenAI

/usr/local/lib/python3.10/site-packages/dash_auth/plotly_auth.py:16: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/usr/local/lib/python3.10/site-packages/dash_auth/plotly_auth.py:17: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [3]:
# Call data preprocessing function to preprocess data
data = None  # Set global_data to None to force a reload of data

### 2- LAUNCH DASH 

In [4]:
#web.open_new_tab('http://127.0.0.1:8090/')
def open_browser():
    web.open_new("http://localhost:{}".format(8090))

In [5]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.UNITED], meta_tags=[{"name": "viewport", "content": "width=device-width"}])
app.title = 'Dashboard'
auth = dash_auth.BasicAuth(app, VALID_USERNAME_PASSWORD_PAIRS)

In [6]:
# Create a Flask-Caching object
cache = Cache(app.server, config={
    'CACHE_TYPE': 'filesystem',
    'CACHE_DIR': 'cache-directory',
    'CACHE_DEFAULT_TIMEOUT': 86400  # 24 hours
})

In [7]:
cache.clear()

True

In [8]:
# Create a dcc.Store component to store the data
store = dcc.Store(id='local', storage_type='local')
# Create a dcc.Store component to store the histo_data
#histo_store = dcc.Store(id='histo_data', storage_type='local')

### 4- PREPROCESS CLEAN DATA

In [9]:
# define a function to preprocess the data
def preprocess_data(df):
    df = df[['sales_order_id','date','id','created_at','quantity','sku','name','name_i','face_price','currency', 
                    'country','state_i', 'state_o','status','external_unit_price', 'name_o',
                   'city', 'country_o']]
    df["created_at"] = df["created_at"].astype('datetime64[ns]')
    df["date"] = pd.to_datetime(df["date"])
    df['date'] = df['date'].fillna(df['created_at'])
    df[['year', 'month', 'day', 'weekday']] = df['created_at'].apply(lambda x: pd.Series([x.year, x.month, x.day, x.day_name()]))
    df[['quantity',  'id']] = df[['quantity','id']].astype('Int64')
    #df[['product_id', 'customer_company_id']] = df[[ 'product_id', 'customer_company_id']].astype('Int64')
    df['face_price'] = df['face_price'].astype('float').round(2)
    df['price'] = df['external_unit_price'].str.extract('(\d+)').astype('float') / 100
    df['price'] = df['price'].round(2)
    df['total'] = (df['quantity'] * df['price']).round(2)
    df['net_price'] = df['face_price'] - df['price']
    df['net_income'] = (df['quantity'] * df['net_price']).round(2) 
    df['percent_margin'] = (df['net_price']/df['face_price']*100).astype('float').round(2)
    df = df[~df['state_i'].isin(['deleted', 'cancelled'])]
    df.sort_values(['created_at'],ascending=[True])
    return df



In [10]:
#load data for current data
def load_current():
    csv_files = ['current.csv', 'currentsql.csv']
    latest_file = max(csv_files, key=os.path.getmtime)
    data = pd.read_csv(latest_file)
    clean_data = preprocess_data(data)
    return clean_data

In [11]:
# load_current().tail(1)

In [12]:
# Set up the database connection
def get_database_connection():
    connection = pymysql.connect(
        host = '127.0.0.1',
        user = 'readonly',
        password = 'MwLvD9DFL8mnrTKkI6fU',
        port = 3335
        #DB_NAME = 'mydatabase',
    )
    return connection

In [13]:
# Read data from the database and store it in csv
def load_current_sql():
    connection = get_database_connection()
    cursor = connection.cursor()
    last_date = "2023-01-01"
    last_date_o = "2023-01-01"
    query = f"""
        SELECT i.sales_order_id, i.id AS id, i.quantity, i.product_id, i.state AS state_i, i.status, i.created_at, i.external_unit_price,
                       p.id AS sku_id, p.sku, p.name, p.description, p.currency, p.brand_id, p.face_price, p.country, b.name AS name_i,
                       o.date, o.invoice_number, o.state AS state_o, o.company_currency_id, cc.company_id AS cust_id, cc.display_name AS name_o, a.city, a.country AS country_o, x.finance_number, x.sales_order_id AS sales_order_id_o
                        FROM (SELECT *
                              FROM ezscm_production.sales_order_items
                              WHERE created_at > '{last_date}')  AS i
                        LEFT OUTER JOIN (SELECT *
                                         FROM ezscm_production.sales_orders
                                         WHERE date > '{last_date_o}') AS o
                            ON o.id = i.sales_order_id
                        LEFT OUTER JOIN ezscm_production.products AS p
                            ON p.id = i.product_id
                        LEFT OUTER JOIN ezscm_production.brands AS b
                            ON p.brand_id = b.id
                        LEFT OUTER JOIN ezscm_production.company_currencies AS cc
                            ON o.company_currency_id = cc.id
                        LEFT OUTER JOIN ezscm_production.companies AS c
                            ON cc.company_id = c.id
                        LEFT OUTER JOIN ezscm_production.addresses AS a
                            ON c.billing_address_id = a.id
                        LEFT OUTER JOIN ezscm_production.invoices AS x
                            ON o.invoice_number = x.finance_number
                            
                            ORDER BY id ASC

                """
    cursor.execute(query)
    rows = cursor.fetchall()
    data = pd.read_sql(query, connection)
    clean_df = preprocess_data(data)
    clean_df.to_csv('currentsql.csv', index=False)
    #cache.set('data', df.to_dict('records'))
    connection.close()
    #return pd.DataFrame.from_dict(df)
    return clean_df


In [14]:
# load_current_sql().tail(1)

### 4.a - FETCH OPTIONS

In [15]:
@app.callback(Output('local', 'data'),
              Output("loading-fetch-data", "children"),
              Input('fetch-data-button', 'n_clicks'))
def fetch_data(n_clicks):
    if n_clicks is None or n_clicks == 0:
        # Try to load data from the cache or CSV file
        stored_data = cache.get('data')
        if stored_data is not None:
            # Use the data from the cache if available
            return stored_data, ''
        else:
            # Otherwise, load the most recent data from the CSV files
            clean_data = load_current()
            cache.set('data', clean_data.to_json(date_format='iso', orient='split'))
            if clean_data.empty:
                return '', html.Div([
                    html.P('Data not found. Click on "Fetch Data" button.')
                ])
            else:
                return clean_data.to_json(date_format='iso', orient='split'), ''  # clean_data.to_dict('records')
    else:
        # Fetch data from the database and convert to pandas DataFrame
        try:
            clean_df = load_current_sql()
            cache.set('data', clean_df.to_json(date_format='iso', orient='split'))
            return clean_df.to_json(date_format='iso', orient='split'), html.Div([
                html.P('Data fetched successfully')
            ])
        except Exception as e:
            return '', html.Div([
                html.P('Connect to MySQL please', style={'color': 'red'})
            ])

In [16]:
# fetch_data(1)

In [17]:
# df = pd.read_json(fetch_data(0)[0], orient='split')
# df.tail(1)

In [18]:
# pd.DataFrame.from_dict(fetch_data(1)[0][-5:])

In [19]:
# json_data = cache.get('data')

# # Get size of the data in bytes
# size_bytes = len(json_data.encode('utf-8'))
# # Convert size to human-readable format (e.g. "1.23 MB")
# size_str = '{:.2f} {}'.format(size_bytes / 1024 / 1024, 'MB')
# size_bytes, size_str


### 5- DISPLAY RESPONSE 

In [20]:
@app.callback(Output('table-container', 'children'),
              [Input('fetch-data-button', 'n_clicks'),
               State('local', 'data')])
def update_data(n_clicks, data):
    if n_clicks is None or n_clicks == 0:
        json_resp = fetch_data(0)[0]
    else:
        json_resp = fetch_data(1)[0]
    # Convert the JSON data to a DataFrame
    df = pd.read_json(json_resp, orient='split')
    # Get the last row of the DataFrame
    last_row = df.tail(1)
   # Return the table with the last row of data
    return dbc.Container([
        html.H6('Last row of Data Table'),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in last_row.columns],
            data=last_row.to_dict('records'),
            style_cell={'textAlign': 'center'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    ])


In [21]:
# update_data(0, data)

In [22]:
# update_data(1, data)

In [23]:
# data_dict = cache.get('data')
# data_df = pd.read_json(data_dict, orient='split')
# data_df.tail(1)

In [24]:
# import sys
# size = sys.getsizeof(data_dict)
# print(f"Size of data_dict: {size} bytes")


In [25]:
def load_cached_data():
    cached_data = cache.get('data')
    if cached_data is not None:
        df = pd.read_json(cached_data, orient='split')
    else: 
        df = load_current()
    return df

In [26]:
# load_cached_data().tail(1)

In [27]:
# len(load_cached_data()['state_i']=='fulfilled')

In [28]:
# len(load_cached_data()['state_o']=='complete')

In [29]:
def load_histo():
    df = load_cached_data()
    if df is None:
        return pd.DataFrame()
    past_data = pd.read_csv('pastfour.csv')
    clean_past = preprocess_data(past_data)
    histo_df = pd.concat([clean_past, df]).drop_duplicates(keep='first').reset_index(drop=True)
    histo_df["created_at"] = histo_df["created_at"].astype('datetime64[ns]')
    histo_df = histo_df[histo_df['state_i'] == 'fulfilled']
    return histo_df

In [30]:
# load_histo().tail(1)

In [31]:
# load_histo()['year'].unique()

In [32]:
# load_histo()['year'].unique()[0]

In [33]:
# @app.callback(Output('histo_data', 'data'),
#               Input('fetch-data-button', 'n_clicks'),
#               State('local', 'data'))
# def update_histo_data(n_clicks, data):
#     if n_clicks is None or n_clicks == 0:
#         json_resp = fetch_data(0)[0]
#     else:
#         json_resp = fetch_data(1)[0]
#     df = pd.read_json(json_resp, orient='split')
#     histo_df = load_histo()
#     return histo_df.to_json(date_format='iso', orient='split')


In [34]:
#code for the pandasai search

In [35]:
#result layout 
search_container = dbc.Container([
    dbc.Row([
        dbc.Col(dcc.Input(id="input", type="text", placeholder="Ex: Which sku is the most in demand in 2023?", n_submit=1, style={'width': '100%'}), width=9),
        dbc.Col(dbc.Button('Search', id='search-button', className='btn-primary'), width=2),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='output-graph'), width=12, id='output-graph-container', style={'display': 'none'})
    ])
], fluid=True, style={'padding': '2rem'})

In [36]:
@app.callback(
    Output("output-graph-container", "style"),
    Output("output-graph", "figure"),
    Input("search-button", "n_clicks"),
    State("input", "value")
)
def search_output(n_clicks, input_value):
    if n_clicks is None:
        raise PreventUpdate

    df = load_cached_data()
    llm = OpenAI(api_token="***")
    pandas_ai = PandasAI(llm)
    prompt = input_value
    result = pandas_ai.run(df, prompt=prompt)

    # Determine the type of result (figure or text)
    if isinstance(result, go.Figure):
        # Graph is returned
        return {'display': 'block'}, result
    else:
        # Text is returned
        return {'display': 'block'}, html.Div(result)

### 6- NAVBAR LAYOUT 

In [37]:
today_date = datetime.now().strftime("%Y-%m-%d")

In [38]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(html.Img(src="db.png", height="20px"), width="auto", align="center"),
                    dbc.Col(dbc.NavbarBrand("EZi Dashboard", className="ml-2"), width="auto", align="center"),
                    dbc.Col(html.Div(id="today-date", children=today_date), width="auto", align="center"),
                    dbc.Col(
                        [
                            dbc.Button('Fetch Data', id='fetch-data-button', n_clicks=0, className="mr-2"),
                     dcc.Loading(
                                id="loading-fetch-data",
                                type="default",
                                children=[
                                    dbc.Button(id='query-status-button', children='',style={'width': 'auto', 'margin-left': '10px'})
                                ]
                            ),
                            html.Div(id='status'),
                        ],
                        width="auto", align="end",style={'display': 'flex', 'align-items': 'center'}
                    ),
                    #html.Button(id='page-load', n_clicks=0, style={'display': 'none'}),
                    html.Div(id='page-load', style={'display': 'none'}, children='page-load'),
                ],
                className="my-row",
                align="center",
            ),
            dbc.NavbarToggler(id="navbar-toggler"),
        ]
    ),
    color="light",
    dark=False,
    sticky="top",
)

In [39]:
# #what percentage of not complete orders do we have
# unsuccess = df[df['state_o']!='complete']
# unsuccess.groupby('name_o').agg({'quantity':'count','total':'sum'}).sort_values(by='quantity', ascending= False).head(5)

In [40]:
# df.tail()

### 8- PAGE BODY : TOP LEVEL VISUALS

In [41]:
# Define page layout
page_layout = dbc.Container([
    
    # First row
    dbc.Row([
        dbc.Col(
            dcc.DatePickerSingle(
                id='date-picker',
                min_date_allowed=None,
                max_date_allowed=None,
                initial_visible_month=None,
                date=None
            ),
            width=1,
            className='align-self-start'
        ),
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("Overnight Orders, including not Complete"),
                    dbc.CardBody(html.P(id='overnight-sales'))
                ],
                className='mb-4',
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            width=5,
            className='align-self-start'
        ),
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("SKU in demand O/N (qty, net income)"),
                    dbc.CardBody(
                        dcc.Graph(
                            id='weighted-proportion-graph',
                            style={'height': '100%'}  # Set the graph height to 100% of the card
                        )
                    )
                ],
                className='mb-4',
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            width=5,
            className='align-self-start'
        ),
    ], justify="between", align="center", className='mb-4', style={'align-items': 'stretch'})
    ,

    # Second row
    dbc.Row([
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("Sales YtD"),
                    dbc.CardBody(dcc.Graph(id='sales-chart-2'))
                ],
                className="h-100",
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            sm=12, lg=4, className="mb-4"
        ),
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("Sales Mtd"),
                    dbc.CardBody(dcc.Graph(id='sales-chart'))
                ],
                className="h-100",
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            sm=12, lg=4, className="mb-4"
        ),
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader(html.H6("Trending SKUs YTD")),
                    dbc.CardBody(dcc.Graph(id='trending-skus-graph'))
                ],
                className="h-100",
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            sm=12, lg=4, className="mb-4"
        )
    ]),

    # Third row
    dbc.Row([
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("Clients that are no longer ordering", style={'backgroundColor': '#F8F9FA'}),
                    dbc.CardBody(
                        dcc.Graph(
                            id='table',
                            config={'displayModeBar': False},
                            style={'padding': '0', 'margin': '0'}
                        )
                    )
                ],
                className="h-100",
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            sm=12, lg=6, className="mb-4"
        ),
        dbc.Col(
            dbc.Card(
                [
                    dbc.CardHeader("Item from last year that are no longer ordered"),
                    dbc.CardBody(dcc.Graph(id='sku-profit-last-year'))
                ],
                className="h-100",
                style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
            ),
            sm=12, lg=6, className="mb-4"
        )
    ]),


    # Add rest of page layout here
    html.Div("Click on below tabs for further analysis 👇"),

], fluid=True, style={'padding': '2rem'})


In [42]:
@app.callback(
    Output('date-picker', 'min_date_allowed'),
    Output('date-picker', 'max_date_allowed'),
    Output('date-picker', 'initial_visible_month'),
    Output('date-picker', 'date'),
    Input('local', 'data'),
    #State('date-picker', 'date'),
)
def update_datepicker(data):
    df = load_cached_data()
    min_date_allowed = df['date'].astype('datetime64[ns]').min().strftime('%Y-%m-%d')
    max_date_allowed = df['date'].astype('datetime64[ns]').max().strftime('%Y-%m-%d')
    initial_visible_month = min_date_allowed
    date = df['date'].astype('datetime64[ns]').max().strftime('%Y-%m-%d')

    return min_date_allowed, max_date_allowed, initial_visible_month, date

In [43]:
# update_datepicker(data)

In [44]:
# data = load_cached_data()

In [45]:
# today = pd.Timestamp.now().floor('D')
# def prev_weekday(adate):
#     adate -= timedelta(days=1)
#     while adate.weekday() > 4: # Mon-Fri are 0-4
#         adate -= timedelta(days=1)
#     return adate

In [46]:
# prev_weekday(today)

In [47]:
@app.callback(
    Output('overnight-sales', 'children'),
    [Input('date-picker', 'date'),
    State('local','data')],)
def update_overnight_sales(date, data):
    df = load_cached_data()
    # Filter dataframe based on selected date
    filtered_df = df[df['date'] >= pd.Timestamp(date).floor('D')]

    # Calculate overnight sales based on selected date
    overnight_sales = round(filtered_df['net_income'].sum(), 2)
    overnight_sales = "{:,.2f}".format(overnight_sales)
    overnight_gross = round(filtered_df['total'].sum(), 2)
    overnight_gross = "{:,.2f}".format(overnight_gross)
    overnight_orders = filtered_df['sales_order_id'].nunique()
    no_not_complete = filtered_df[filtered_df['state_o'] != 'complete']['sales_order_id'].nunique()
    so_not_complete = filtered_df[filtered_df['state_o'] != 'complete']['sales_order_id'].unique()
    # links = [f"[{so}](https://ezi.ezcards.xyz/sales_orders/{so})" for so in so_not_complete]
    # so_not_complete_str = ', '.join(links)
    not_complete_df = filtered_df[filtered_df['state_o'] != 'complete']
    so_info = []
    
    for so_number in not_complete_df['sales_order_id'].unique():
        so_data = not_complete_df[not_complete_df['sales_order_id'] == so_number]
        net_income = so_data['net_income'].sum()
        link = f"[{so_number}](https://ezi.ezcards.xyz/sales_orders/{so_number})"
        so_info.append(f"**SO: {so_number}** - Net Income: ${net_income:.2f} ({link})")

    return html.Div([
        f"O/N activity = $ {overnight_gross} (Gross), $ {overnight_sales} (Net), from {overnight_orders} orders, where {len(not_complete_df['sales_order_id'].unique())} order(s) not fulfilled : ",
        html.Div([
            dcc.Markdown(info) for info in so_info
        ])
    ])

    # return html.Div([
    #     f"O/N activity = $ {overnight_gross} gross, $ {overnight_sales} net, from {overnight_orders} orders, where {no_not_complete} order(s) not fulfilled : ",
    #     html.Div(dcc.Markdown(so_not_complete_str))
    # ])


In [48]:
# update_overnight_sales(prev_weekday(today), data)

In [49]:
@app.callback(
    Output('weighted-proportion-graph', 'figure'),
    [Input('date-picker', 'date')],
    [State('local', 'data')],
)
def update_weighted_proportion_graph(date, data):
    df = load_cached_data()
    # Filter dataframe based on selected date
    filtered_df = df[df['date'] >= pd.Timestamp(date).floor('D')]

    # Group by SKU and calculate sum of quantity and net income
    sku_group = filtered_df.groupby('sku').agg({'quantity': 'sum', 'net_income': 'sum'}).reset_index()
    
    # Sort SKUs by quantity in descending order
    sku_group = sku_group.sort_values('quantity', ascending=False)

    # Get top 7 and bottom 3 SKUs
    top_7_skus = sku_group.head(10)
    #bottom_3_skus = sku_group.tail(3)

    # Create bar chart for top 7 and bottom 3 SKUs
    fig = go.Figure()

    # Add top 7 SKUs
    fig.add_trace(go.Bar(
        x=top_7_skus['sku'],
        y=top_7_skus['quantity'],
        name='Top 10 SKUs (qty, net $ sold)',
        text=top_7_skus['net_income'],
        textposition='auto',
        marker_color='deepskyblue'  # Change color to deepskyblue
    ))

    # # Add bottom 3 SKUs
    # fig.add_trace(go.Bar(
    #     x=bottom_3_skus['sku'],
    #     y=bottom_3_skus['quantity'],
    #     name='Bottom 3 SKUs',
    #     text=bottom_3_skus['net_income'],
    #     textposition='auto',
    #     marker_color='#db5f23'  
    # ))

    #Set chart layout
    fig.update_layout(
        # xaxis_title='SKU',
        # yaxis_title='Quantity',
        barmode='group',
        legend=dict(
            orientation='h',  # Place legend below x-axis horizontally
            yanchor='bottom',
            y = -0.1
        )
    )

    return fig

In [50]:
# update_weighted_proportion_graph(prev_weekday(today), data)

In [51]:
def new_sales_chart(date, df):
    current_year = pd.Timestamp(date).year
    current_month = pd.Timestamp(date).month
    current_day = pd.Timestamp(date).day
    prev_year = current_year - 1
    two_year = current_year - 2

    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    last_ytd_start = pd.to_datetime(f"{prev_year}-01-01")
    last_ytd_end = pd.to_datetime(f"{prev_year}-{current_month}-{current_day}")
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")
    two_ytd_end = pd.to_datetime(f"{two_year}-{current_month}-{current_day}")

    sales_ytd_total = round(df[(df['date'] >= ytd_start)]['total'].sum(), 2)
    sales_ytd_net_income = round(df[(df['date'] >= ytd_start)]['net_income'].sum(), 2)
    sales_ytd_total_formatted = "{:,.2f}".format(sales_ytd_total)
    sales_ytd_net_income_formatted = "{:,.2f}".format(sales_ytd_net_income)
    sales_ytd_last_year_total = df[(df['date'] >=last_ytd_start)& (df['date'] <= last_ytd_end)]['total'].sum()
    sales_ytd_last_year_net_income = df[(df['date'] >=last_ytd_start)& (df['date'] <= last_ytd_end)]['net_income'].sum()
    sales_ytd_two_year_total = df[(df['date'] >=two_ytd_start)& (df['date'] <= two_ytd_end)]['total'].sum()
    sales_ytd_two_year_net_income = df[(df['date'] >=two_ytd_start)& (df['date'] <= two_ytd_end)]['net_income'].sum()
    percent_vs_last_year_total_to_date = round(sales_ytd_total/sales_ytd_last_year_total*100, 2)
    percent_vs_last_year_net_income_to_date = round(sales_ytd_net_income/sales_ytd_last_year_net_income*100, 2)
    percent_vs_two_year_total_to_date = sales_ytd_total/sales_ytd_two_year_total
    percent_vs_two_year_net_income_to_date = sales_ytd_net_income/sales_ytd_two_year_net_income

    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=percent_vs_last_year_total_to_date,
        gauge={
            'axis': {'range': [None, 100], 'tickwidth': 1, 'tickcolor': 'darkblue'},
            'bar': {'color': 'deepskyblue'},
            'bgcolor': 'white',
            'borderwidth': 2,
            'bordercolor': 'gray',
            'steps': [{'range': [0, 50], 'color': 'white'},
                      {'range': [50, 100], 'color': 'white'},
                      {'range': [100, 150], 'color': 'white'},
                      {'range': [1500, 200], 'color': 'white'}]
        },
        domain={'x': [0, 1], 'y': [0, 1]}
    ))
    
    fig.update_layout(
        annotations=[
            go.layout.Annotation(
                x=0.5,
                y=1.2,
                showarrow=False,
                text=f"YTD Gross Sales: $ {sales_ytd_total_formatted}",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=16)
            ),
            go.layout.Annotation(
                x=0.5,
                y=1.1,
                showarrow=False,
                text=f"YTD Net sales: $ {sales_ytd_net_income_formatted}",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=16)
            ),
            go.layout.Annotation(
                x=0.5,
                y=-0.1,
                showarrow=False,
                text=f"{percent_vs_last_year_total_to_date}% done vs past YTD sales (Gross)",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=16)
            ),
            go.layout.Annotation(
                x=0.5,
                y=-0.2,
                showarrow=False,
                text=f"{percent_vs_last_year_net_income_to_date}% done vs past YTD sales (Net)",
                xref="paper",
                yref="paper",
                align='center',
                font=dict(size=16)
            )
        ]
    )
    
    return fig

In [52]:
# new_sales_chart(prev_weekday(today), data)

In [53]:
@app.callback(
    Output('sales-chart-2', 'figure'),
    [Input('date-picker', 'date')],
    [State('local', 'data')]
)

def updated_new_sales_chart(date, data):
    current_year = pd.Timestamp(date).year
    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    past_year = current_year - 1
    two_year = current_year - 2
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")
    
    # Load the data based on whether the CSV file exists or not
    if load_histo() is None:
        df = load_cached_data()
    else:
        df = load_histo()

    # Group the data by month and sum the sales
    fig = new_sales_chart(date, df)
    
    return fig

In [54]:
# updated_new_sales_chart(prev_weekday(today), data)

In [55]:
# # displaying bar chart for month to date
# def create_sales_chart(df, date):
#     month = pd.Timestamp(date).to_pydatetime()
#     sales_mtd = df[(df['month'] == month.month) & (df['year'] == month.year)]['total'].sum()
#     sales_mtd_formatted = "{:,.2f}".format(sales_mtd)
#     sales_mtd_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['total'].sum()
#     sales_mtd_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['total'].sum()
#     percent_vs_last_year_month = round(sales_mtd/sales_mtd_last_year*100,2)
#     percent_vs_two_year_month = round(sales_mtd/sales_mtd_two_year*100,2)

#     labels = ['vs 2Y ago', 'vs last year','MtD']
#     category_names = [sales_mtd_two_year, sales_mtd_last_year, sales_mtd_formatted]

#     trace = go.Bar(
#         x=category_names,
#         y=labels,
#         orientation='h',
#         text=category_names,
#         textposition='auto',
#         marker=dict(
#             color='deepskyblue'
#         )
#     )

#     data = [trace]
#     layout = go.Layout(
#         #xaxis=dict(title='Total Sales'),
#         #yaxis=dict(title='Category'),
#         margin=dict(l=1)
#     )

#     fig = go.Figure(data=data, layout=layout)
#     return fig


In [56]:
def create_sales_chart(df, date):
    month = pd.Timestamp(date).to_pydatetime()
    sales_mtd = df[(df['month'] == month.month) & (df['year'] == month.year)]['total'].sum()
    sales_mtd_net = df[(df['month'] == month.month) & (df['year'] == month.year)]['net_income'].sum()
    sales_mtd_formatted = "{:,.2f}".format(sales_mtd)
    sales_mtd_net_formatted = "{:,.2f}".format(sales_mtd_net)
    
    sales_mtd_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['total'].sum()
    sales_mtd_net_last_year = df[(df['month'] == month.month) & (df['year'] == month.year - 1)]['net_income'].sum()
    sales_mtd_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['total'].sum()
    sales_mtd_net_two_year = df[(df['month'] == month.month) & (df['year'] == month.year - 2)]['net_income'].sum()
    
    sales_mtd_last_year_formatted = "{:,.2f}".format(sales_mtd_last_year)
    sales_mtd_net_last_year_formatted = "{:,.2f}".format(sales_mtd_net_last_year)
    sales_mtd_two_year_formatted = "{:,.2f}".format(sales_mtd_two_year)
    sales_mtd_net_two_year_formatted = "{:,.2f}".format(sales_mtd_net_two_year)
    
    gross_minus_net = sales_mtd - sales_mtd_net
    gross_minus_net_last_year = sales_mtd_last_year - sales_mtd_net_last_year
    gross_minus_net_two_years = sales_mtd_two_year - sales_mtd_net_two_year
    
    percent_vs_last_year_month = round(sales_mtd/sales_mtd_last_year*100, 2)
    percent_vs_two_year_month = round(sales_mtd/sales_mtd_two_year*100, 2)

    labels = ['vs 2Y ago', 'vs last year', 'MtD']
    category_names = [sales_mtd_two_year, sales_mtd_last_year, sales_mtd_formatted]

    net = go.Bar(
        x=[sales_mtd_net_two_year, sales_mtd_net_last_year, sales_mtd_net],
        y=labels,
        orientation='h',
        text=[sales_mtd_net_two_year_formatted, sales_mtd_net_last_year_formatted, sales_mtd_net_formatted],
        textposition='auto',
        marker=dict(
            color='deepskyblue'
        )
    )

    gross = go.Bar(
        x=[gross_minus_net_two_years, gross_minus_net_last_year , gross_minus_net],
        y=labels,
        orientation='h',
        text=[sales_mtd_two_year_formatted, sales_mtd_last_year_formatted, sales_mtd_formatted],
        textposition='auto',
        marker=dict(
            color='lightgray'
        )
    )

    data = [net,gross]
    layout = go.Layout(
        xaxis=dict(title='Total Sales'),
        yaxis=dict(title='Category'),
        margin=dict(l=1),
        barmode='stack',
        showlegend=False
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

In [57]:
@app.callback(
    Output('sales-chart', 'figure'),
    [Input('date-picker', 'date')],
    [State('local', 'data')]
)
def update_sales_chart(date, data):
    # Load the data based on whether the histo file exists or not
    if load_histo() is None:
        df = load_cached_data()
    else:
        df = load_histo()
    sales_chart = create_sales_chart(df, date)
    return sales_chart

In [58]:
# create_sales_chart(data, prev_weekday(today))

In [59]:
# list of customers that did not create an order since...
def get_customers_to_chase(df):
    # Filter for customers who last ordered this year
    current_year = datetime.now().year
    df.name_o.dropna()
    # How long have we got each cust
    customer_length = df.groupby('name_o').agg({'sales_order_id': 'nunique', 'created_at': ['min', 'max']})
    customer_length['longevity'] = customer_length['created_at']['max'] - customer_length['created_at']['min']
    customer_length = customer_length.reset_index()
    customer_length.columns = ['name_o', 'orders_count', 'first_order', 'last_order', 'longevity']
    customer_length['last_order'] = pd.to_datetime(customer_length['last_order'], format='%Y-%m-%d')
    filtered_df = customer_length.loc[(customer_length['last_order'].dt.year == current_year)]
    sorted_df = filtered_df[['name_o', 'orders_count', 'last_order', 'longevity']].sort_values('last_order').head(10)
    # Format the longevity column in years and months
    sorted_df['longevity'] = sorted_df['longevity'].apply(lambda x: f'{int(x.days/365)}y {int((x.days%365)/30)}m')
    sorted_df['last_order'] = pd.to_datetime(sorted_df['last_order'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')

    table = go.Figure(data=[go.Table(
        columnwidth=1,
        header=dict(values=list(sorted_df.columns),
                    fill_color='lightgray',
                    align=['left', 'center']),
        cells=dict(values=[sorted_df.name_o, sorted_df.orders_count, sorted_df.last_order, sorted_df.longevity],
                   fill_color='white',
                   align=['left', 'center']))
    ])

    return table

In [60]:
# @app.callback(
#     Output('table', 'figure'),
#     [Input('local', 'data')],
#     #prevent_initial_callback=True
# )
# def update_table(data):
#     df = load_cached_data()
#     table = get_customers_to_chase(df)
#     return table

In [61]:
@app.callback(
    Output('table', 'figure'),
    Input('local', 'data')
)
def update_table(data):
    df = load_cached_data()
    table = get_customers_to_chase(df)
    return table.to_dict() 

In [62]:
@app.callback(
    Output('trending-skus-graph', 'figure'),
    [Input('date-picker', 'date')],
    [State('local', 'data')]
)
def update_trending_skus_graph(date, data):
    df = load_cached_data()

    # Group by SKU and calculate the occurrence across orders, net income, and average discount
    sku_group = df.groupby('sku').agg({
        'id': 'count',  # Occurrence across orders
        'net_income': 'sum',
        'percent_margin': 'mean'  # Average discount
    }).reset_index()

    # Filter out negative values of net_income
    sku_group = sku_group[sku_group['net_income'] > 0]

    # Sort SKUs by net income in descending order
    sku_group = sku_group.sort_values('net_income', ascending=False)

    # Select the top 25 SKUs
    top_25_skus = sku_group.head(25)

    # Create violin plot for trending SKUs
    fig = px.scatter(top_25_skus, x='id', y='net_income', color='sku', size='percent_margin', hover_data=['percent_margin'])
    fig.update_layout(
        xaxis_title='demand',
        yaxis_title='profit',
        #title='Top 25 SKUs - Trend Analysis',
        showlegend=True
    )

    return fig

In [63]:
def find_missing_skus(date, data):
    current_year = pd.Timestamp(date).year
    ytd_start = pd.to_datetime(f"{current_year}-01-01")
    past_year = current_year - 1
    two_year = current_year - 2
    two_ytd_start = pd.to_datetime(f"{two_year}-01-01")

    # Load the data based on whether the CSV file exists or not
    if load_histo() is None:
        df = load_cached_data()
    else:
        df = load_histo()

    last_year_orders = df[(df['date'] >= pd.to_datetime(f"{past_year}-01-01")) & (df['date'] < ytd_start)][['sku', 'percent_margin']]
    current_year_orders = df[(df['date'] >= ytd_start)][['sku', 'percent_margin']]
    missing_skus = []

    for sku in last_year_orders['sku'].unique():
        if sku not in current_year_orders['sku'].unique():
            avg_margin = last_year_orders[last_year_orders['sku'] == sku]['percent_margin'].mean().round(2)
            if avg_margin <= 40.0:
                missing_skus.append({'sku': sku, 'avg_margin': avg_margin})

    missing_skus = sorted(missing_skus, key=lambda x: x['sku'])
    return missing_skus

In [64]:
# find_missing_skus(prev_weekday(today), data)

In [65]:
@app.callback(
    Output('sku-profit-last-year', 'figure'),
    Input('date-picker', 'date')
)
def update_profit_last_year_graph(date):
    avg_margin_dict = find_missing_skus(date, data)
    df = pd.DataFrame(avg_margin_dict)
    df = df.sort_values('avg_margin', ascending=False)

    fig = go.Figure(
        data=[
            go.Bar(x=df['sku'], y=df['avg_margin'], marker_color='deepskyblue')
        ],
        layout=go.Layout(
            xaxis_title='SKU',
            yaxis_title='Average Margin',
            barmode='group'
        )
    )

    return fig

In [66]:
# update_profit_last_year_graph(prev_weekday(today))

In [67]:
# load_cached_data()

In [68]:
# update_table(data)

### 9- FIRST TAB - SALES DASHBOARD

In [69]:
# @app.callback(Output('histo-data', 'data'), [Input('page-load', 'children')])
# def update_histo_data(children):
#     df = load_histo() if load_histo() is not None else load_cached_data()
#     #print(df.columns)  # print the column names of the dataframe
#     return df.to_dict('records')

In [70]:
# update_histo_data()

In [71]:
# @app.callback(
#     Output('year-dropdown', 'options'),
#     [Input('local', 'data')]
# )
# def update_year_dropdown(data):
#     histo_df = load_histo()
#     years = histo_df['year'].dropna().unique()
#     options = [{'label': str(year), 'value': year} for year in years]
#     return options

In [72]:
# update_year_dropdown(data)

In [73]:
# update_year_dropdown(data)[-1]

In [74]:
current_year = datetime.now().year

In [75]:
# Sales dashboard
sales_tab_content = dbc.Container([
     dbc.Row([
        dbc.Col(html.H6('... discover below ...', className='card-title mb-4')),
        html.Label('Select year:'),
        dcc.Dropdown(
            id='year-dropdown',
            clearable=False,
            searchable=False,
            placeholder='Select a year',
            #options=[],
            options=[{'label': year, 'value': year} for year in range(2019, 2024)],
            value=current_year,
        ),
        ], justify="center", align="center", className='mb-4'),

    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Top 3 Customers")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-top-3-cust')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Average Order Value")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='avg-order-value')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Sales by Region")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sales-map')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        # dbc.Col([
        #     dbc.Card([
        #         dbc.CardHeader(html.H5("Sales vs Order vs Total SKU")),
        #         dbc.CardBody(
        #             [
        #                 dcc.Graph(id='sales-order-sku')
        #             ],
        #             style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
        #         ),
        #     ], 
        #     style={'height': '100%'}
        #     )
        # ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Profit generated by")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='profit-chart')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
], fluid=True, style={'padding': '2rem'})

In [76]:
@app.callback(
    Output('sales-top-3-cust', 'figure'),
    [Input('year-dropdown', 'value')],
    [State('local', 'data')],
    prevent_initial_callback=True
)
def update_sales_graph_1(selected_year, data):
    # Load the data based on whether the histo file exists or not
    df = load_cached_data() if selected_year == current_year else load_histo()

    # Filter the data by the selected year
    filtered_df = df[df['year'] == selected_year]

    # Calculate total sales by month and customer
    sales_by_month_customer = filtered_df.groupby(['month', 'name_o'])['total'].sum().reset_index()

    # Find top customers for each month
    total_sales_by_customer = df.groupby('name_o')['total'].sum()
    top_customers = total_sales_by_customer.nlargest(20).index.tolist()
    customer_colors = {customer: px.colors.qualitative.Pastel1[i % len(px.colors.qualitative.Pastel1)]
                   for i, customer in enumerate(top_customers)}
    customer_colors['others'] = 'gray'
    def assign_color(customer):
        if customer in customer_colors:
            return customer_colors[customer]
        else:
            return 'gray'
    top_customers_by_month = sales_by_month_customer.groupby('month').apply(lambda x: x.nlargest(3, 'total')).reset_index(drop=True)

    # Group the remaining customers as "others"
    remaining_customers_by_month = sales_by_month_customer[~sales_by_month_customer['name_o'].isin(top_customers)]
    remaining_customers_by_month = remaining_customers_by_month.groupby('month').sum().reset_index()
    remaining_customers_by_month['name_o'] = 'others'

    # Concatenate the top customers and the "others" dataframes
    sales_by_month_customer = pd.concat([top_customers_by_month, remaining_customers_by_month])

    # Assign colors to each customer using the color mapping dictionary
    sales_by_month_customer['color'] = sales_by_month_customer['name_o'].apply(assign_color)

    # Create stacked bar chart of sales by customer
    fig = px.bar(sales_by_month_customer, x='month', y='total', color='name_o', color_discrete_map=customer_colors)
    fig.update_layout(
        title=f'Top 3 customers and others by month ({selected_year})',
        xaxis_title='Month',
        yaxis_title='Total Sales ($)',
        barmode='stack'
    )
    return fig


In [77]:
# selected_year_2 = df.year.unique()[0]
# selected_year_2

In [78]:
@app.callback(
    Output('avg-order-value', 'figure'),
    [Input('year-dropdown', 'value')],
    [State('local','data')],
    prevent_initial_callback=True
)
def update_avg_order_value(selected_year, data):
    # Load the data based on whether the histo file exists or not
    df = load_cached_data() if selected_year == current_year else load_histo()

    # Filter the data by the selected year
    filtered_df = df[df['year'] == selected_year]

    # Calculate the number of orders, total order value, and average order value by client
    avg_order_value_by_client = filtered_df.groupby(['name_o']).agg({'total': 'sum', 'sales_order_id': 'nunique'})
    avg_order_value_by_client['avg_order_value'] = avg_order_value_by_client['total'] / avg_order_value_by_client['sales_order_id']

    # Calculate the number of items per basket by client
    items_per_basket_by_client = filtered_df.groupby(['name_o', 'sales_order_id'])['sku'].nunique().reset_index().groupby(['name_o'])['sku'].mean().reset_index()
    items_per_basket_by_client = items_per_basket_by_client.rename(columns={'sku': 'items_per_basket'})

    # Calculate the average percent off face price sold by client
    avg_item_percent_off_by_client = filtered_df.groupby(['name_o'])['percent_margin'].mean().reset_index()

    # Merge all the dataframes together
    merged_df = pd.merge(avg_order_value_by_client, items_per_basket_by_client, on='name_o')
    merged_df = pd.merge(merged_df, avg_item_percent_off_by_client, on='name_o')

    # Handle missing or invalid values in percent_margin
    merged_df['percent_margin'] = merged_df['percent_margin'].fillna(0)  # Replace missing values with 0
    merged_df['percent_margin'] = merged_df['percent_margin'].clip(lower=0)  # Set negative values to 0

    # Create the figure
    fig = px.scatter(merged_df, x='avg_order_value', y='items_per_basket', size='percent_margin', color='name_o', hover_name='name_o')
    fig.update_layout(
        title=f'Average Order Value and Items per Basket by Client ({selected_year})',
        xaxis_title='Average Order Value ($)',
        yaxis_title='Items per Basket'
    )
    return fig

In [79]:
@app.callback(
    Output('sales-map', 'figure'),
    [Input('year-dropdown', 'value')],
    [State('local', 'data')],
    prevent_initial_callback=True
)
def update_sales_map(selected_year, data):
    # Load the data based on whether the histo file exists or not
    df = load_cached_data() if selected_year == current_year else load_histo()

    # Filter the data by the selected year
    filtered_df = df[df['year'] == selected_year]

    # Perform country code mapping to country names
    sales_by_region = filtered_df.groupby(['country_o']).agg({'total': 'sum'}).reset_index()
    #sales_by_region['country_iso_alpha3'] = sales_by_region['country_o'].apply(lambda code: pycountry.countries.get(alpha_2=code).alpha_3)
    sales_by_region['country_iso_alpha3'] = sales_by_region['country_o'].apply(lambda code: pycountry.countries.get(alpha_2=code).alpha_3 if pycountry.countries.get(alpha_2=code) else None)

    fig = px.choropleth(sales_by_region, locations='country_iso_alpha3', color='total',
                        projection='natural earth')
    fig.update_layout(title=f'Total Sales by Region ({selected_year})')
    return fig

In [80]:
@app.callback(
    Output('profit-chart', 'figure'),
    [Input('year-dropdown', 'value')],
    [State('local', 'data')],
    prevent_initial_callback=True
)
def update_profit_chart(selected_year, data):
    # Load the data based on whether the histo file exists or not
    df = load_cached_data() if selected_year == current_year else load_histo()

    # Filter the DataFrame
    filtered_df = df[df['state_o'] == 'complete']

    # Group the data and calculate the sum of net_income
    grouped_df = filtered_df.groupby(['name_o', 'sku']).agg({'net_income': 'sum'}).reset_index()

    # Find the top 3 clients per year
    top_clients = grouped_df.groupby('name_o')['net_income'].sum().nlargest(5).index.tolist()
    grouped_df['name_top'] = grouped_df['name_o'].apply(lambda x: x if x in top_clients else 'Others')

    # Sort the SKU names in alphabetical order
    sorted_skus = sorted(grouped_df['sku'].unique())

    # Create the bar chart
    fig = px.bar(grouped_df, x='name_top', y='net_income', color='sku', title=f'Profit Generated by Items and Clients ({selected_year})',
                 category_orders={'sku': sorted_skus},
                hover_data={'name_o': True, 'sku': True})
    fig.update_layout(xaxis_title='Client', yaxis_title='Profit', legend_title='Item')

    return fig


In [81]:
# @app.callback(
#     Output('sales-order-sku', 'figure'),
#     [Input('year-dropdown', 'value')],
#     [State('local','data')],
#     #prevent_initial_callback=True
# )
# def update_sales_graph_2(selected_year,data):
#     # Load the data based on whether the histo file exists or not
#     df = load_cached_data() if selected_year == current_year else load_histo()

#     filtered_df = df[df['year'] == selected_year]
#     # Total Sales trend by month
#     sales_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M')).sum()['total']
#     #sales_by_month = filtered_df.groupby(filtered_df['date'].dt.to_period('M'))['total'].sum()
#     sales_trace = go.Scatter(x=sales_by_month.index, y=sales_by_month.values, mode='lines', name='Total Sales')

#     # Total Unique Orders trend by month
#     unique_orders_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['id'].nunique()
#     #unique_orders_by_month = filtered_df.groupby(filtered_df['date'].dt.to_period('M'))['id'].nunique()
#     unique_orders_trace = go.Scatter(x=unique_orders_by_month.index, y=unique_orders_by_month.values, mode='lines', name='Total Unique Orders')

#     # Total SKU trend by month
#     total_sku_by_month = filtered_df.groupby(pd.Grouper(key='date', freq='M'))['quantity'].sum()
#     #total_sku_by_month = filtered_df.groupby(filtered_df['date'].dt.to_period('M'))['quantity'].sum()
#     total_sku_trace = go.Scatter(x=total_sku_by_month.index, y=total_sku_by_month.values, mode='lines', name='Total SKUs')
    
#     fig = go.Figure()
    
#     fig.add_trace(sales_trace)
#     fig.add_trace(unique_orders_trace)
#     fig.add_trace(total_sku_trace)

#     fig.update_layout(
#         font=dict(size=10),
#         margin=dict(l=20, r=20, t=40, b=20),
#         paper_bgcolor="white",
#         plot_bgcolor="white",
#         xaxis_title='Month',
#         yaxis_title='Total',
#         title="Sales, Unique Orders, and SKUs Trend"
#     )
    
#     # # Set the x-axis tick labels to show all the months
#     # fig.update_xaxes(
#     #     type='category',
#     #     tickmode='array',
#     #     tickvals=sales_by_month.index,
#     #     ticktext=[calendar.month_name[m.month] for m in sales_by_month.index],
#     #     dtick='M1'
#     # )

#     return fig
# # update_sales_graph_2(2021)

### 10- SECOND TAB : CLIENT DASHBOARD

In [82]:
@app.callback(
    Output('client-dropdown', 'options'),
    Input('local', 'data')
)
def update_client_dropdown_options(data):
    df = load_cached_data()  # Assuming this function loads the data from cache
    if df is not None:
        unique_names = df['name_o'].dropna().unique()
        unique_names_sorted = sorted(unique_names)
        options = [{'label': name, 'value': name} for name in unique_names_sorted]
        return options
    return []


In [83]:
# Create the second tab content for clients
clients_tab_content = dbc.Container([
     dbc.Row([
        dbc.Col(html.H6('... it will be filled with insights on phase 2 ...', className='card-title mb-4')),
        html.Label('Pick a name:'),
        dcc.Dropdown(
            id='client-dropdown',
            clearable=False,
            searchable=False,
            placeholder='Pick a client',
            #options=[],
        ),
        ], justify="center", align="center", className='mb-4'),
    dbc.Row(
        dbc.Card(
            [
                dbc.CardHeader(html.H5("Sensitivity to price or season")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='sensi-graph')
                        # dcc.Slider(
                        #     id='bandwidth-slider',
                        #     min=df['bandwidth'].min(),
                        #     max=df['bandwidth'].max(),
                        #     step=0.1,
                        #     value=df['bandwidth'].min(),
                        #     marks={str(bandwidth): str(bandwidth) for bandwidth in df['bandwidth'].unique()}
                        # )
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column', 'overflow-y': 'scroll'}
                ),
            ],
            className="w-100",
        ),
    ),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Client Favourite SKU (Top 3)")),
                dbc.CardBody(
                    [
                        #dcc.Graph(id='client-favourite-sku-graph')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("SKU Bundled Together")),
                dbc.CardBody(
                    [
                        #dcc.Graph(id='sku-bundled-output')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Client Periodicity of Order")),
                dbc.CardBody(
                    [
                        #dcc.Graph(id='client-periodicity-graph')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H5("Orders per Customer by Year")),
                dbc.CardBody(
                    [
                        #dcc.Graph(id='orders-per-customer-heatmap')
                    ], 
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column'}
                ),
            ], 
            style={'height': '100%'}
            )
        ], md=6),
    ]),
], fluid=True, style={'padding': '2rem'})

### sensivity to price or season

In [84]:
@app.callback(
    Output("sensi-graph", "figure"),
    Input('local', 'data')
)
def update_sensitivity_graph(data):  # Replace with your actual input value parameter
    # Filter the DataFrame based on the input value or any other relevant criteria
    #filtered_df = df[df["name_o"] == 'BASATNE INTERNATIONAL LLC']  # Replace with your actual filtering condition
    df = load_cached_data() #if selected_year == current_year else load_histo()
    # Group the filtered DataFrame by 'name_o' and calculate the mean sensitivity scores
    grouped_df = df.groupby("name_o").mean()
    
    # Create the plotly figure
    fig = go.Figure(
        data=[
            go.Scatter(
                x=grouped_df["sensitivity_season_score"],
                y=grouped_df["sensitivity_price_score"],
                mode="markers",
                marker=dict(size=10),
                text=grouped_df.index,
                hovertemplate="Client: %{text}<br>"
                "Sensitivity to Season: %{x}<br>"
                "Sensitivity to Price: %{y}<extra></extra>",
            )
        ],
        layout=go.Layout(
            title="Client Sensitivity Analysis",
            xaxis=dict(title="Sensitivity to Season"),
            yaxis=dict(title="Sensitivity to Price"),
            hovermode="closest",
        ),
    )

    return fig


In [85]:
# @app.callback(
#     Output('client-favourite-sku-graph', 'figure'),
#     Input('client-favourite-sku-dropdown', 'value')
# )
# def update_client_favourite_sku_graph(client_name):
#     res = dfa[dfa['name_o'].str.contains(client_name)].groupby('name_o')['sku'].value_counts(normalize=True)
#     labels = res.index.get_level_values(1).unique().tolist()
#     fig = go.Figure(data=[go.Pie(labels=labels, values=res, hole=0.3)])
#     return fig


In [86]:
# def plot_periodicity(df, client):
#     # filter data for given client
#     client_data = df[df['name_o'] == client]
    
#     # calculate sales by weekday, month, and year
#     weekdays = client_data.groupby(['name_o', 'weekday']).agg({'total': 'sum'})
#     months = client_data.groupby(['name_o', 'month']).agg({'total': 'sum'})
#     years = client_data.groupby(['name_o', 'year']).agg({'total': 'sum'})
    
#     # create bar chart for sales by weekday, month, and year
#     fig = px.bar(x=weekdays.index.get_level_values('weekday'), y=weekdays['total'], labels={'x': 'Weekday', 'y': 'Sales'})
#     fig.update_layout(title=f"Sales by weekday for {client}")
    
#     fig2 = px.bar(x=months.index.get_level_values('month'), y=months['total'], labels={'x': 'Month', 'y': 'Sales'})
#     fig2.update_layout(title=f"Sales by month for {client}")
    
#     fig3 = px.bar(x=years.index.get_level_values('year'), y=years['total'], labels={'x': 'Year', 'y': 'Sales'})
#     fig3.update_layout(title=f"Sales by year for {client}")
    
#     return fig, fig2, fig3

In [87]:
# @app.callback(
#     [Output(component_id='periodicity-by-weekday', component_property='figure'),
#      Output(component_id='periodicity-by-month', component_property='figure'),
#      Output(component_id='periodicity-by-year', component_property='figure')],
#     [Input(component_id='client-dropdown', component_property='value')]
# )
# def update_periodicity_figures(client):
#     fig, fig2, fig3 = plot_periodicity(df, client)
#     return fig, fig2, fig3

### 11- THIRD TAB : PROSPECT DASHBOARD

In [88]:
prospects_tab_content = dbc.Container([
    dbc.Row(
        dbc.Col(html.H6('... it will be filled with insights on phase 2 ...', className='card-title mb-4'),
                #width={'size': 6, 'offset': 3}
               ),
        justify="center",
        align="center",
        className='mb-4'
    ),
    dbc.Row(
        dbc.Card(
            [
                dbc.CardHeader(html.H5("In a nutshell")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='habits-graph')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column', 'overflow-y': 'scroll'}
                ),
            ],
            className="w-100",
        ),
    ),
], fluid=True, style={'padding': '2rem'})


In [89]:
@app.callback(
    Output("habits-graph", "figure"),
    Input("local", "data")
)
def update_habits_graph(data):
    df = load_histo()
    customer_orders = df.groupby(['name_o', 'year', 'month', 'weekday']).agg({
            'sales_order_id': 'nunique',
            'date': ['min', 'max']
        })
    customer_orders = customer_orders.reset_index()
    customer_orders.columns = ['name_o', 'year', 'month', 'weekday', 'orders_count', 'first_order', 'last_order']

    years = customer_orders.groupby(['name_o', 'year']).agg({'orders_count': 'sum'})
    year_pivot = years.pivot_table(index='name_o', columns='year', values='orders_count')

    fig = go.Figure(data=go.Heatmap(
        z=year_pivot.values,
        x=year_pivot.columns,
        y=year_pivot.index,
        colorscale='Blues',
        #nan_color='grey',
        colorbar=dict(title='Orders Count')
    ))

    fig.update_layout(
        title='Orders per Customer by Year',
        xaxis=dict(title='Year', tickangle=45),
        yaxis=dict(title='Customer'),
        height=3000,
        margin=dict(l=20, r=20, t=40, b=20)
    )

    return fig


### 12- FOURTH TAB : ITEM DEMAND PRICING AND SEASONALITY

In [90]:
@app.callback(
    Output('item-dropdown', 'options'),
    Input('local', 'data')
)
def update_item_dropdown_options(data):
    df = load_cached_data()  # Assuming this function loads the data from cache
    if df is not None:
        unique_names = df['sku'].dropna().unique()
        unique_names_sorted = sorted(unique_names)
        options = [{'label': name, 'value': name} for name in unique_names_sorted]
        return options
    return []


In [91]:
items_tab_content = dbc.Container([
    dbc.Row([
        dbc.Col(html.H6('... it will be filled with insights on phase 2 ...', className='card-title mb-4')),
        html.Label('Pick an item:'),
        dcc.Dropdown(
            id='item-dropdown',
            clearable=False,
            searchable=False,
            placeholder='Pick an item',
            # options=[],
        )
    ], justify="center", align="center", className='mb-4'),
    dbc.Row(
        dbc.Card(
            [
                dbc.CardHeader(html.H5("Product demand and price")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='item-graph')
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column', 'overflow-y': 'scroll'}
                ),
            ],
            className="w-100",
        ),
    ),
], fluid=True, style={'padding': '2rem'})

In [92]:
@app.callback(
    Output("item-graph", "figure"),
    [Input("item-dropdown", "value")],
    [State('local','data')]
)
def update_items_graph(selected_item,data):
    df = load_cached_data()
    
    # Filter the dataframe based on the selected item
    filtered_df = df[df['sku'] == selected_item]
    
    # Group by month and year, and calculate the average price per month
    avg_prices = filtered_df.groupby(['month', 'year'])['price'].mean().reset_index()
    
    fig = go.Figure(data=go.Scatter(
        x=avg_prices['month'].astype(str) + '-' + avg_prices['year'].astype(str),
        y=avg_prices['price'],
        mode='lines',
        name=selected_item,
        line=dict(color='blue')
    ))

    layout = go.Layout(
        title='Price Evolution for {}'.format(selected_item),
        xaxis=dict(title='Month'),
        yaxis=dict(title='Price'),
        legend=dict(orientation='h'),
        height=500,
        margin=dict(l=20, r=20, t=40, b=20)
    )

    fig.update_layout(layout)

    return fig


### 12- FIFTH TAB : SALES PREDICTION

In [93]:
# @app.callback(
#     Output('item-dropdown', 'options'),
#     Input('local', 'data')
# )
# def update_item_dropdown_options(data):
#     df = load_cached_data()  # Assuming this function loads the data from cache
#     if df is not None:
#         unique_names = df['sku'].dropna().unique()
#         unique_names_sorted = sorted(unique_names)
#         options = [{'label': name, 'value': name} for name in unique_names_sorted]
#         return options
#     return []


In [94]:
proj_tab_content = dbc.Container([
    dbc.Row([
        dbc.Col(html.H6('... it will be filled with insights on phase 2 ...', className='card-title mb-4')),
    ], justify="center", align="center", className='mb-4'),
    dbc.Row(
        dbc.Card(
            [
                dbc.CardHeader(html.H5("Sales prediction")),
                dbc.CardBody(
                    [
                        dcc.Graph(id='proj-graph')
                        # dcc.Slider(
                        #     id='bandwidth-slider',
                        #     min=df['bandwidth'].min(),
                        #     max=df['bandwidth'].max(),
                        #     step=0.1,
                        #     value=df['bandwidth'].min(),
                        #     marks={str(bandwidth): str(bandwidth) for bandwidth in df['bandwidth'].unique()}
                        # )
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column', 'overflow-y': 'scroll'}
                ),
            ],
            className="w-100",
        ),
    ),
], fluid=True, style={'padding': '2rem'})

In [95]:
@app.callback(
    Output('proj-graph', 'figure'),
    #[Input('bandwidth-slider', 'value')],
    [Input('local','data')]
)
def update_sales_graph(bandwidth, data):
    # Select the relevant data based on the chosen bandwidth
    df = load_cached_data()
    data = df[df['bandwidth'] == bandwidth]

    # Split the data into features (X) and target (y)
    X = data['bandwidth'].values.reshape(-1, 1)
    y = data['sales'].values

    # Create a Linear Regression model
    model = LinearRegression()

    # Fit the model to the data
    model.fit(X, y)

    # Use the model to make predictions
    y_pred = model.predict(X)

    # Create the plotly figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=X.flatten(),
        y=y,
        mode='markers',
        name='Actual'
    ))
    fig.add_trace(go.Scatter(
        x=X.flatten(),
        y=y_pred,
        mode='lines',
        name='Predicted'
    ))
    fig.update_layout(
        xaxis=dict(title='Promotion Bandwidth'),
        yaxis=dict(title='Sales'),
        title='Sales prediction based on Promotion Bandwidth'
    )

    return fig


### 12ter- SIXTH TAB : BEHAVIOUR ANALYSIS

In [96]:
bs_tab_content = dbc.Container([
    dbc.Row([
        dbc.Col(html.H6('... it will be filled with insights on phase 2 ...', className='card-title mb-4')),
    ], justify="center", align="center", className='mb-4'),
    dbc.Row(
        dbc.Card(
            [
                dbc.CardHeader(html.H5("Multivariate analysis")),
                dbc.CardBody(
                    [
                        html.Div([
                            dcc.Dropdown(
                                id='x-axis',
                                options=[],
                                value='weekday',
                                clearable=False
                            ),
                            dcc.Dropdown(
                                id='y-axis',
                                options=[],
                                value='quantity',
                                clearable=False
                            ),
                        ], style={'width': '50%', 'margin': 'auto'}),
                        dcc.Graph(id='scatter-plot'),
                    ],
                    style={'height': '100%', 'display': 'flex', 'flex-direction': 'column', 'overflow-y': 'scroll'}
                ),
            ],
            className="w-100",
        ),
    ),
], fluid=True, style={'padding': '2rem'})

In [97]:
# Callback to update the dropdown options
@app.callback(
    [Output('x-axis', 'options'), Output('y-axis', 'options')],
    [Input('local', 'data')]
)
def update_dropdown_options(data):
    if data:
        df1 = pd.read_json(data, orient='split')
        options = [{'label': col, 'value': col} for col in df1.columns]
        return options, options
    return [], []
# def update_dropdown_options(data):
#     if data:
#         df1 = pd.read_json(data, orient='split')
#         grouping_functions = {
#             'count': 'Count',
#             'sum': 'Sum',
#             'mean': 'Mean'
#         }

#         options = [{'label': col, 'value': col} for col in df1.columns]
#         options_with_grouping = [{'label': f'{grouping_functions[func]}({col})', 'value': f'{func}:{col}'} for col in df1.columns for func in grouping_functions]

#         options = options_with_grouping
#         return options, options
#     return [], []


In [98]:
# Define the callback function
@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('x-axis', 'value'), Input('y-axis', 'value')],
    [State('local', 'data')]
)
def update_scatter_plot(x_axis, y_axis, data): #for only multivariate
    if data:
        df1 = pd.read_json(data, orient='split')
        fig = px.scatter(
            data_frame=df1,
            x=x_axis,
            y=y_axis,
            title=f'Relationship between {x_axis} and {y_axis}',
            height=500,
            color='name_o'
        )
        return fig
    return {}
# def update_scatter_plot(x_axis, y_axis, data):
#     if data:
#         df1 = pd.read_json(data, orient='split')

#         def apply_grouping(data_frame, value):
#             if ':' in value:
#                 grouping_func, col = value.split(':')
#                 if grouping_func == 'count':
#                     return data_frame.groupby(col).size().reset_index(name='count')
#                 elif grouping_func == 'sum':
#                     return data_frame.groupby(col).sum().reset_index()
#                 elif grouping_func == 'mean':
#                     return data_frame.groupby(col).mean().reset_index()
#             return data_frame

#         df_grouped = df1
#         df_grouped = apply_grouping(df_grouped, x_axis)
#         df_grouped = apply_grouping(df_grouped, y_axis)

#         fig = px.scatter(
#             data_frame=df_grouped,
#             x=x_axis,
#             y=y_axis,
#             title=f'Relationship between {x_axis} and {y_axis}',
#             height=500,
#             color='name_o'
#         )

#         return fig

#     return {}


### 13- GLOBAL LAYOUT > END

In [99]:
# Define tabs
tabs = dbc.Tabs(
    [
        dbc.Tab(label="Historic Sales", children=sales_tab_content),
        dbc.Tab(label="Customer Analysis", children=clients_tab_content),
        dbc.Tab(label="Prospects Market Research", children=prospects_tab_content),
        dbc.Tab(label="Product price demand ratios", children=items_tab_content),
        dbc.Tab(label="Sales prediction", children=proj_tab_content),
        dbc.Tab(label="Behaviour analysis", children=bs_tab_content),
    ],
    id="tabs",
    active_tab="Historic Sales",
)

In [100]:
# Define app layout
app.layout = html.Div([
    store, #dcc.Store(id='local', storage_type='local'),  # Move dcc.Store outside navbar
    navbar,
    html.Div(id='table-container'),
    search_container,
    page_layout,
    dbc.Container([
        dbc.Row([
            dbc.Col([
                tabs,
            ]),
        ], className='mt-4')
    ])
], style={'padding': '0rem 0rem 4rem 0rem'})

In [101]:
# Set the app layout
#app.layout = app_layout
if __name__ == '__main__':
    Timer(1, open_browser).start();
    app.run_server(debug=True, use_reloader=False, port=8090)

Dash is running on http://127.0.0.1:8090/

 * Serving Flask app '__main__'
 * Debug mode: on


/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_74577/3210813908.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_74577/3210813908.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jt/qpx02c196772_ytczgpv4llh0000gp/T/ipykernel_74577/3210813908.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d